# Team Assignment


1.   Srinath Mannam (229750)
2.   Meghana Rao (234907)
3.   Govind Shukla (235192)

# import statements

In [1]:
import os
import tensorflow as tf
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
from numba import cuda
import sys
import tensorflow.keras.backend as K
from tensorflow.python.keras.utils.vis_utils import model_to_dot
from IPython.display import SVG
import pydot
import graphviz
from keras.callbacks import ModelCheckpoint
from datetime import datetime
from keras.preprocessing import sequence

# Change the directory

In [2]:
working_directory = '/content/drive/My Drive/Colab Notebooks/OVGU/Deep_Learning/06_Assignment'
def colabDrive():
    from google.colab import drive
    drive.mount("/content/drive")

    if os.getcwd() !=  working_directory:
      os.chdir(working_directory)
    print(os.getcwd())

In [3]:
colabDrive()

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/OVGU/Deep_Learning/06_Assignment


# Prepare the data

## Load the dataset and remove infrequent words

In [4]:
num_words = 40000
(train_sequences, train_labels), (test_sequences, test_labels) = tf.keras.datasets.imdb.load_data(num_words=num_words)

def preprocess(sequences, labels):
    return sequences, labels.astype(np.int32)

train_sequences, train_labels = preprocess(train_sequences, train_labels)
test_sequences, test_labels = preprocess(test_sequences, test_labels)

print('Loaded dataset with {} training samples, {} test samples'.format(len(train_sequences), len(test_sequences)))

17473536/17464789 [==============================] - 1s 0us/step
Loaded dataset with 25000 training samples, 25000 test samples


## to restore words, load the word-to-index mapping and look at one example

In [5]:
word2id = tf.keras.datasets.imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}
print('---review with words---')
print([id2word.get(i, ' ') for i in train_sequences[6]])
print('---label---')
print(train_sequences[6])
print('---label---')
print(train_labels[6])

1654784/1641221 [==============================] - 0s 0us/step
---review with words---
['the', 'boiled', 'full', 'involving', 'to', 'impressive', 'boring', 'this', 'as', 'murdering', 'naschy', 'br', 'villain', 'council', 'suggestion', 'need', 'has', 'of', 'costumes', 'b', 'message', 'to', 'may', 'of', 'props', 'this', 'echoed', 'concentrates', 'concept', 'issue', 'skeptical', 'to', "god's", 'he', 'is', 'and', 'unfolds', 'movie', 'women', 'like', "isn't", 'surely', "i'm", 'rocketed', 'to', 'toward', 'in', "here's", 'for', 'from', 'did', 'having', 'because', 'very', 'quality', 'it', 'is', "captain's", 'starship', 'really', 'book', 'is', 'both', 'too', 'worked', 'carl', 'of', 'and', 'br', 'of', 'reviewer', 'closer', 'figure', 'really', 'there', 'will', 'originals', 'things', 'is', 'far', 'this', 'make', 'mistakes', "kevin's", 'was', "couldn't", 'of', 'few', 'br', 'of', 'you', 'to', "don't", 'female', 'than', 'place', 'she', 'to', 'was', 'between', 'that', 'nothing', 'dose', 'movies', 'get

## Maximum review length and minimum review length.

In [6]:
print('Maximum review length: {}'.format(len(max((train_sequences + test_sequences), key=len))))

Maximum review length: 2697


In [7]:
print('Minimum review length: {}'.format(len(min((train_sequences + test_sequences), key=len))))

Minimum review length: 70


## Generator to create different shaped train,test dataset

In [8]:
def gen_Train():
    for sequence, label in zip(train_sequences, train_labels):
        yield sequence, label
      
def gen_Test():
    for sequence, label in zip(test_sequences, test_labels):
        yield sequence, label

## Create training, test dataset tensors

In [9]:
def createData(bucketFlag):
  train_data = tf.data.Dataset.from_generator(gen_Train, output_signature=(
          tf.TensorSpec(shape=(None,), dtype=tf.int32),
          tf.TensorSpec(shape=(), dtype=tf.int32)))

  test_data = tf.data.Dataset.from_generator(gen_Test, output_signature=(
          tf.TensorSpec(shape=(None,), dtype=tf.int32),
          tf.TensorSpec(shape=(), dtype=tf.int32)))
  
  # Do bucket padding if the flag is true or do normal padding
  if bucketFlag:
    buckets = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500]
    bucket_batch_size = [32] * (len(buckets) + 1)
    train_data = train_data.bucket_by_sequence_length(lambda sequence, label: tf.shape(sequence)[0],
                                                      bucket_boundaries=buckets, bucket_batch_sizes=bucket_batch_size)
    test_data = test_data.bucket_by_sequence_length(lambda sequence, label: tf.shape(sequence)[0],
                                                      bucket_boundaries=buckets, bucket_batch_sizes=bucket_batch_size)
    return (train_data, test_data)
    
  train_data = train_data.padded_batch(32)
  test_data = test_data.padded_batch(32)
  return (train_data, test_data)

In [10]:
(train_data, test_data) = createData(True)

In [11]:
print("Train sequences Batches")
for step, (sequence, label) in enumerate(train_data):
    print(sequence.shape, label.shape)

    if step is 3:
      break

print("\nTest sequences Batches")
for step, (sequence, label) in enumerate(test_data):
    print(sequence.shape, label.shape)

    if step is 3:
      break

Train sequences Batches
(32, 147) (32,)
(32, 198) (32,)
(32, 1038) (32,)
(32, 240) (32,)

Test sequences Batches
(32, 141) (32,)
(32, 197) (32,)
(32, 149) (32,)
(32, 249) (32,)


# Generic Functions

## Training Loop

In [12]:
def model_Train(curr_Model, EPOCHS=5):
  temp_Model = curr_Model

  temp_Model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
  
  print("Traing Start")
  for epoch in range(EPOCHS):
    losses, accs = [], []
    print("Epoch : ", epoch+1)
    for step, (seq_batch, label_batch) in enumerate(train_data):
      metrics = temp_Model.train_on_batch(seq_batch, label_batch, return_dict=True)

      losses.append(metrics['loss'])
      accs.append(metrics['accuracy'])
      if not step % 250:
        print("Step No: {}, Loss: {}, Acc: {}".format(step, np.mean(losses), np.mean(accs)))
        losses, accs = [], []
  print("Traing End\n")

  return (temp_Model)

## Testing Loop

In [13]:
def model_Test(curr_Model):

  curr_Model.summary()

  for step, (seq_batch, label_batch) in enumerate(test_data):
    metrics = curr_Model.test_on_batch(seq_batch, label_batch, reset_metrics=False, return_dict=True)
  print("Test loss: {} and Acc: {}".format(metrics['loss'], metrics['accuracy']))

# Models

## Training

### RNN

In [14]:
model_rnn = tf.keras.Sequential(
                [
                tf.keras.layers.Embedding(input_dim=num_words, output_dim=32, mask_zero=True),
                tf.keras.layers.RNN(tf.keras.layers.SimpleRNNCell(128)),
                tf.keras.layers.Dense(1)
                ]
            )
  
model_rnn = model_Train(model_rnn, EPOCHS=10)

model_rnn.save('saved_models/model_rnn_epochs10_bucketPad_V1.h5') 

Traing Start
Epoch :  1
Step No: 0, Loss: 0.7167055010795593, Acc: 0.6875
Step No: 250, Loss: 0.6951793818473816, Acc: 0.4945
Step No: 500, Loss: 0.6912703089714051, Acc: 0.502125
Step No: 750, Loss: 0.6909706101417542, Acc: 0.502625
Epoch :  2
Step No: 0, Loss: 0.6690125465393066, Acc: 0.6875
Step No: 250, Loss: 0.6793604576587677, Acc: 0.4975
Step No: 500, Loss: 0.6792122764587403, Acc: 0.505125
Step No: 750, Loss: 0.6752134323120117, Acc: 0.50275
Epoch :  3
Step No: 0, Loss: 0.6531992554664612, Acc: 0.6875
Step No: 250, Loss: 0.6526642587184907, Acc: 0.503
Step No: 500, Loss: 0.6470011608600617, Acc: 0.52275
Step No: 750, Loss: 0.6467424151897431, Acc: 0.528125
Epoch :  4
Step No: 0, Loss: 0.5807011723518372, Acc: 0.75
Step No: 250, Loss: 0.5961750588417053, Acc: 0.58425
Step No: 500, Loss: 0.578988703250885, Acc: 0.6315
Step No: 750, Loss: 0.5635873651504517, Acc: 0.6465
Epoch :  5
Step No: 0, Loss: 0.4601864218711853, Acc: 0.84375
Step No: 250, Loss: 0.4627342277765274, Acc: 0.782

### LSTM

In [ ]:
model_rnn = tf.keras.Sequential(
                [
                tf.keras.layers.Embedding(input_dim=num_words, output_dim=32, mask_zero=True),
                tf.keras.layers.LSTM(12),
                tf.keras.layers.Dense(1)
                ]
            )
  
model_rnn = model_Train(model_rnn, EPOCHS=10)

model_rnn.save('saved_models/model_lstm_epochs10_bucketPad_V1.h5') 

### Bi-directional RNN

In [ ]:
model_brnn = tf.keras.Sequential(
                [
                tf.keras.layers.Embedding(input_dim=num_words, output_dim=32, mask_zero=True),
                tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(12)),
                tf.keras.layers.Dense(64, activation='relu'),
                tf.keras.layers.Dense(1)
                ]
            )
  
model_brnn = model_Train(model_rnn, EPOCHS=10)

model_brnn.save('saved_models/model_brnn_epochs10_bucketPad_V1.h5') 

## Testing

In [26]:
models_Path = {
    'rnn_models': ['saved_models/model_rnn_epochs10_bucketPad_V1.h5'],
    'lstm_models': ['saved_models/model_lstm_epochs10_bucketPad_V1.h5'],
    'blstm_models': ['saved_models/model_blstm_epochs10_bucketPad_V1.h5' ]
}

In [30]:
for key in models_Path:
  print("----------------------------------Model Type----------------------------------------\n")
  print("Currently Testing the {} \n".format(key))
  for ind, val in enumerate(models_Path[key]):
    print("Model Summary \n")
    loaded_model = tf.keras.models.load_model(val)
    print("Model Test Accuracy \n")
    model_Test(loaded_model)
    print("\n\n")

----------------------------------Model Type----------------------------------------

Currently Testing the rnn_models 

Model Summary 

Model Test Accuracy 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          1280000   
                                                                 
 rnn (RNN)                   (None, 128)               20608     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,300,737
Trainable params: 1,300,737
Non-trainable params: 0
_________________________________________________________________
Test loss: 0.48918139934539795 and Acc: 0.8226400017738342



----------------------------------Model Type----------------------------------------

Cur